<a href="https://colab.research.google.com/github/MDC55/Basic-Deep-Learning/blob/main/Hsi_Fahim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.ehu.eus/ccwintco/index.php/Hyperspectral_Remote_Sensing_Scenes#Indian_Pines
#https://github.com/DharmeshPatel33/Land-Cover-Classification-using-Hyper-Spectral-Data-with-Deep-Learning/blob/main/CNN_Hybrid_Spectral_Net.ipynb

In [ ]:
!pip install spectral
!pip install keras.utils

In [ ]:
import os
import scipy.io as sio
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical as keras_to_categorical
#from keras.utils import np_utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(os.path.join('/content/drive/MyDrive/','HSI_Dataset'))

/content/drive/MyDrive/HSI_Dataset


In [ ]:
def loadData(name):
    data_path = os.path.join('/content/drive/MyDrive/','HSI_Dataset')
    if name == 'IP':
        data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['paviaU']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['paviaU_gt']

    return data, labels

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
## GLOBAL VARIABLES
dataset = 'IP'
test_ratio = 0.7
windowSize = 25

In [ ]:
X, y = loadData(dataset)

X.shape, y.shape

((145, 145, 200), (145, 145))

In [ ]:
K = X.shape[2]

In [ ]:
K = 30 if dataset == 'IP' else 15
X,pca = applyPCA(X,numComponents=K)

X.shape

(145, 145, 30)

In [ ]:
X, y = createImageCubes(X, y, windowSize=windowSize)

X.shape, y.shape

((10249, 25, 25, 30), (10249,))

In [ ]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((3074, 25, 25, 30), (7175, 25, 25, 30), (3074,), (7175,))

##Model and Training

In [ ]:
Xtrain = Xtrain.reshape(-1, windowSize, windowSize, K, 1)
Xtrain.shape

(3074, 25, 25, 30, 1)

In [ ]:
ytrain = keras_to_categorical(ytrain)
ytrain.shape

(3074, 16)

In [ ]:
S = windowSize
L = K
output_units = 9 if (dataset == 'PU' or dataset == 'PC') else 16 #number of classes

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import *
from tensorflow import (abs,cast,clip_by_value,complex,concat,convert_to_tensor,expand_dims,gather,gather_nd,linspace,map_fn,matmul,

    norm, pad,print,range,repeat,reshape,shape,sign,split, squeeze, stack, tensor_scatter_nd_update,tile, transpose, unstack, zeros,)

In [ ]:
def normed_activation_block(x):
    x = Activation("gelu")(x)
    return BatchNormalization()(x)


def conv_stem(x, filters: int, patch_size: int):
    x = Conv2D(filters, kernel_size=patch_size, strides=patch_size)(x)
    return normed_activation_block(x)

def channel_attn(ftr_block,filters):
  z = GlobalAveragePooling2D()(ftr_block)
  z = tf.expand_dims(z,1)
  z = tf.expand_dims(z,1)
  z=Conv2D(filters//4, (3,3),padding='same',activation='relu')(z)
  z=Conv2D(filters, (3,3),padding='same',activation='sigmoid')(z)
  mul=Multiply()([z, ftr_block])
  return mul

def mixer_block(x, filters: int, kernel_size: int):
    # Depthwise convolution.
    x0 = x
    y1 = DepthwiseConv2D(kernel_size=1, padding="same")(x)
    y2 = DepthwiseConv2D(kernel_size=3, padding="same")(x)
    y3 = DepthwiseConv2D(kernel_size=5, padding="same")(x)
    x = Add()([normed_activation_block(y1),normed_activation_block(y2),normed_activation_block(y3), x0])  # Residual.
    # Pointwise convolution.
    x = Conv2D(filters, kernel_size=1)(x)
    z1 = channel_attn(x,filters)
    x = Add()([z1, x])
    x = normed_activation_block(x)
    return x


def mixer_256_8(image_size=224, filters=256, depth=8, kernel_size=5, patch_size=2, num_classes=10):
    inputs = keras.Input((image_size, image_size, 3))
    x = Rescaling(scale=1.0 / 255)(inputs)
    # Extract patch embeddings.
    x = conv_stem(x, filters, patch_size)
    # Mixer blocks.
    for _ in range(depth):
        x = mixer_block(x, filters, kernel_size)
    # Classification block.
    x = GlobalAvgPool2D()(x)
    outputs = Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


def mixer_layer(image_ftr, filters=256, depth=8, kernel_size=5, patch_size=2):
    # Extract patch embeddings.
    x = conv_stem(image_ftr, filters, patch_size)
    # Mixer blocks.
    for _ in range(depth):
        x = mixer_block(x, filters, kernel_size)
    x = GlobalAvgPool2D()(x)
    return x


def mixer_ops(image_input, filters=256, depth=8, kernel_size=5, patch_size=2):
    x_low = Conv2D(3, (1,1),padding='same',activation='relu')(image_input)
    x_low = MaxPooling2D()(x_low)
    x_base = Conv2D(3, (3,3),padding='same',activation='relu')(image_input)
    x_high = Conv2D(3, (5,5),padding='same',activation='relu')(image_input)
    x_high = UpSampling2D()(x_high)
    x_low_embd = mixer_layer(x_low,filters, depth, kernel_size, patch_size)
    x_base_embd = mixer_layer(x_base,filters, depth, kernel_size, patch_size)
    x_high_embd = mixer_layer(x_high,filters, depth, kernel_size, patch_size)
    outputs =  concatenate([x_low_embd, x_base_embd, x_high_embd])
    return outputs

**Xtrain r shape: (3074, 25, 25, 30, 1)= (batch, height, width, band, channel)**

**Batch dim r por, extra dim 4 ta, so 3D data but amra are 2D diya porcess korbo**



 **Tar age amra 2D backbone banai, jeta 3D data r jonno babohar kora hoibe.**

 **Sei 2D backbone input jabe Batch dim r por extra 3 ta dims: (H, W, Channel)**

In [ ]:
# First define the 2D model as the trainable beackbone

filters = 64
kernel_size = 5
dropout = 0.3
patch_size = 2
depths = 8


##--------------------------------------------------------------------
Batch_dim, height_dim, width_dim, band_dim, cahnnel_dim = Xtrain.shape



##--------------------------------------------------------------------
inputs = Input((height_dim,width_dim, cahnnel_dim))

cb_y = mixer_ops(inputs, filters, depths, kernel_size, patch_size )
bone_mixer =  Model(inputs=inputs, outputs=cb_y )
bone_mixer.trainable = True

bone_mixer.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 25, 25, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 25, 25, 3)            6         ['input_1[0][0]']             
                                                                                                  
 conv2d_2 (Conv2D)           (None, 25, 25, 3)            78        ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 12, 12, 3)            0         ['conv2d[0][0]']              
 D)                                                                                           

In [ ]:
cb_y

<KerasTensor: shape=(None, 192) dtype=float32 (created by layer 'concatenate')>

**Assuming, the data comes in as ( batch, height, width, band, channel)**

In [ ]:
 #

filters = 64
kernel_size = 3
dropout = 0.3
patch_size = 2
depths = 8

number_class = output_units

h_dmn = height_dim # height

w_dmn = width_dim # width

ch_dmn = cahnnel_dim # chnnel


b_dmn = band_dim # band

cb_dmn = 10 # band_cluster # As, cv_dmn-->b_dmn, model will be slower and inverse is true also. So, its a hyperparameter,
            #and sweetspot is needed for best performance acc/speed.

grps = b_dmn//cb_dmn


inputs = Input((height_dim, width_dim, band_dim, cahnnel_dim ))


outputs = []


for i in range(grps):

    b_clusters = inputs[ :,:,:,i*cb_dmn: (i+1)*cb_dmn,: ]

    outputs.append(  bone_mixer(tf.reduce_sum( b_clusters,3)))

av_band =  concatenate(outputs,1)
av_band = Dropout(rate= 0.5, seed= 123)(av_band)

y = Dense(256, kernel_regularizer= l2(l= 0.016), activation= 'relu') (av_band )
y = Dropout(rate= 0.35, seed= 123)(y)

outputs = Dense(number_class, activation='softmax')(y)


pyra_3d_mixer_model_band_last =  Model(inputs=inputs, outputs=outputs )


pyra_3d_mixer_model_band_last .summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 25, 25, 30, 1)]      0         []                            
                                                                                                  
 tf.__operators__.getitem (  (None, 25, 25, 10, 1)        0         ['input_2[0][0]']             
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.__operators__.getitem_1  (None, 25, 25, 10, 1)        0         ['input_2[0][0]']             
  (SlicingOpLambda)                                                                               
                                                                                            

In [ ]:
# from keras.optimizers import Adam
# # compiling the model
# adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.001, decay=1e-06)
# bone_mixer.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# # checkpoint
# filepath = "gdrive/My Drive/HIS_Final/best-model.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

# history = bone_mixer.fit(x=Xtrain, y=ytrain, batch_size=256, epochs=15, callbacks=callbacks_list)

In [ ]:
print(Xtrain.shape,  ytrain.shape)

(3074, 25, 25, 30, 1) (3074, 16)


In [ ]:
from keras.optimizers import Adam
# compiling the model
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.001, decay=1e-06)
pyra_3d_mixer_model_band_last.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
# checkpoint
filepath = "gdrive/My Drive/HIS_Final/best-model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = pyra_3d_mixer_model_band_last.fit(x=Xtrain, y=ytrain, batch_size=32, epochs=10, callbacks=callbacks_list)

Epoch 1/10
97/97 [==============================] - ETA: 0s - loss: 4.3521 - accuracy: 0.6571

97/97 [==============================] - 118s 293ms/step - loss: 4.3521 - accuracy: 0.6571
Epoch 2/10
97/97 [==============================] - ETA: 0s - loss: 1.3744 - accuracy: 0.9021

97/97 [==============================] - 28s 289ms/step - loss: 1.3744 - accuracy: 0.9021
Epoch 3/10
97/97 [==============================] - ETA: 0s - loss: 0.6830 - accuracy: 0.9408

97/97 [==============================] - 28s 289ms/step - loss: 0.6830 - accuracy: 0.9408
Epoch 4/10
97/97 [==============================] - ETA: 0s - loss: 0.5093 - accuracy: 0.9333

97/97 [==============================] - 28s 290ms/step - loss: 0.5093 - accuracy: 0.9333
Epoch 5/10
97/97 [==============================] - ETA: 0s - loss: 0.3933 - accuracy: 0.9577

...

97/97 [==============================] - 28s 291ms/step - loss: 0.3648 - accuracy: 0.9626
Epoch 7/10
97/97 [==============================] - ETA: 0s - loss: 0.2672 - accuracy: 0.9808

97/97 [==============================] - 28s 291ms/step - loss: 0.2672 - accuracy: 0.9808
Epoch 8/10
97/97 [==============================] - ETA: 0s - loss: 0.3032 - accuracy: 0.9642

97/97 [==============================] - 28s 291ms/step - loss: 0.3032 - accuracy: 0.9642
Epoch 9/10
97/97 [==============================] - ETA: 0s - loss: 0.3356 - accuracy: 0.9684

97/97 [==============================] - 28s 292ms/step - loss: 0.3356 - accuracy: 0.9684
Epoch 10/10
97/97 [==============================] - ETA: 0s - loss: 0.3901 - accuracy: 0.9574

97/97 [==============================] - 28s 292ms/step - loss: 0.3901 - accuracy: 0.9574


In [ ]:

Xtest_1, ytest_1 = np.expand_dims(Xtest,-1),  keras_to_categorical(ytest)
print(Xtest_1.shape, ytest_1.shape)

(7175, 25, 25, 30, 1) (7175, 16)


In [ ]:
loss_v , accuracy_v = pyra_3d_mixer_model_band_last.evaluate(Xtest_1, ytest_1)

225/225 [==============================] - 31s 91ms/step - loss: 0.4490 - accuracy: 0.9268
